Design and implement in code a Planner algorithm for a square chessboard of a given size (MxM) with \
certain cells marked red as restricted areas. 100x100 will be the maximum chessboard size. There are N \
kings on the board, each with a specific target cell to reach. The kings move sequentially, one by one, one \
step at a time (K1 step1, K2 step1…. K1 step2, K2 step2…), and must adhere to two main rules: they \
cannot move into restricted cells, and they cannot end a move adjacent to another king (including \
diagonally). A king may stay in place (empty move). The task is to compute a path for each king to its \
target, ensuring compliance with the movement rules and the sequential order of moves. \
Planner’s input would be given as 2 files: \
File1 - king starting and target positions \
x_from, y_from, x_to, y_to \
… \
File2 - chessboard size and restricted cells \
M \
x, y \
…\
x and y - integer coordinates of restricted cells - start from zero. \
If plan (solution) does not exist, Planner should detect and output that plan does not exist.\
If plan does exist, Planner should output a plan in a format \
x_from, y_from, x_to, y_to \
… \
where each line is one step of one king, and king_id would be deduced by a test program from “from” \
coordinates. 

In [1]:
import numpy as np
import math
import heapq
import os
from copy import deepcopy
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

In [2]:
def get_map(test_case):
    
    map_file = open('test_cases/' + str(test_case) + '/map.txt', 'r')
    map_file = str(map_file.read())
    lines = map_file.strip().split('\n')
    obstacles = [tuple(line.split(',')) for line in lines[1:]]
    obstacles = [(int(obstacle[0]),int(obstacle[1])) for obstacle in obstacles]
    grid_size = (int(lines[0]), int(lines[0]))

    return grid_size, obstacles


def get_kings(test_case):
    '''
    This function returns the start and 
    the goal position of the kings
    as a list of tuples' tuples
    '''
    kings_file = open('test_cases/' + str(test_case) + '/kings.txt', 'r')
    kings_file = str(kings_file.read())
    start_goal = kings_file.strip().split('\n')
    total_no_kings = len(start_goal)

    kings = np.zeros(total_no_kings, dtype=object)

    for i, king in enumerate(start_goal):
        king_i = king.split(',')
        start = (int(king_i[0]),int(king_i[1]))
        goal = (int(king_i[2]),int(king_i[3]))
        kings[i] = (start, goal)

    return kings, total_no_kings

def initialize_env(test_case):

    grid_size, obstacles = get_map(test_case)
    kings, total_no_kings = get_kings(test_case)

    return grid_size, obstacles, kings, total_no_kings

# A* Algorithm

In [3]:
class AStarPathfinder:
    def __init__(self, grid_size, start_pos, goal_pos, obstacles):
        self.grid_size = grid_size
        self.start_pos = start_pos
        self.goal_pos = goal_pos
        self.obstacles = obstacles
        self.grid = np.zeros(grid_size, dtype=np.int16)
        # print(self.grid)
        for obs in obstacles:
            self.grid[obs] = 1  # Mark obstacles on the grid

    def heuristic(self, a, b):
        """Calculate the Manhattan distance from point a to point b."""
        return abs(a[0] - b[0]) + abs(a[1] - b[1])

    def neighbors(self, node):
        """Generate the neighbors of a given node."""
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]  # 4 possible movements
        result = []
        for dx, dy in directions:
            nx, ny = node[0] + dx, node[1] + dy
            if 0 <= nx < self.grid_size[0] and 0 <= ny < self.grid_size[1]:
                if self.grid[nx, ny] == 0:  # Check if it's not an obstacle
                    result.append((nx, ny))
        return result

    def a_star_search(self):
        """Perform the A* search algorithm."""
        open_set = []
        heapq.heappush(open_set, (0 + self.heuristic(self.start_pos, self.goal_pos), self.start_pos))
        came_from = {}
        g_score = {self.start_pos: 0}
        f_score = {self.start_pos: self.heuristic(self.start_pos, self.goal_pos)}

        while open_set:
            current = heapq.heappop(open_set)[1]

            if current == self.goal_pos:
                return self.reconstruct_path(came_from, current)

            for neighbor in self.neighbors(current):
                tentative_g_score = g_score[current] + 1  # step cost is always 1
                if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g_score
                    f_score[neighbor] = tentative_g_score + self.heuristic(neighbor, self.goal_pos)
                    heapq.heappush(open_set, (f_score[neighbor], neighbor))

        return None

    def reconstruct_path(self, came_from, current):
        """Reconstruct the path from start to goal."""
        total_path = [current]
        while current in came_from:
            current = came_from[current]
            total_path.append(current)
        return total_path[::-1]  # Return reversed path

# Usage example
# grid_size = (16, 16)
# start_pos = (1, 3)
# goal_pos = (3, 1)
# obstacles = [(0,2), (2,2), (2,1)]
# pathfinder = AStarPathfinder(grid_size, start_pos, goal_pos, obstacles)
# path = pathfinder.a_star_search()
# print(path)



In [9]:
test_case = 1

## Initializing the Environment
grid_size, obstacles, kings, total_no_kings = initialize_env(test_case)

#kings (ndarray) --> [((start1), (goal1))  ((start2), (goal2)) ... ((startN), (goalN))]
def get_paths(kings, obstacles): 
    # all_paths = np.zeros(total_no_kings, dtype=object)
    all_paths = [None] * total_no_kings
    all_path_costs = np.zeros(total_no_kings, dtype=int)

    for i, king in enumerate(kings):
        pathfinder = AStarPathfinder(grid_size, king[0], king[1], obstacles)
        path = pathfinder.a_star_search()
        all_path_costs[i] = len(path)
        all_paths[i] = path
    
    solution_cost = np.sum(all_path_costs)
    
    return all_paths, solution_cost

# print(get_paths(kings, obstacles))

# Conflict-Based Search (High Level)

In [11]:

def get_constraints(all_paths):
    '''
       This function give a dictionary of 
       conflicts for each king
       with other kings
    '''

    constraints = {}           #structure {agent : [list of indices of]}
    for k in range(total_no_kings):  
        constraints[k] = []

    for agent, path in enumerate(all_paths):
        temp = np.delete(all_paths, agent, 0)   #deleting the path, of which vertices are being checked
        for p in path:     # p is each vertex of the path
            for t, path_i in enumerate(temp): 
                if p in path_i:
                    collision = path_i.index(p)
                    agent_with_constraint = all_paths.index(path_i)               
                    constraints[agent].append((agent_with_constraint, collision))
    
    return constraints

# print('conflict', conflict)              

In [12]:
init_paths, init_SIC = get_paths(kings, obstacles)   #SIC --> Sum of Idividual Costs
constraints = get_constraints(init_paths)
# for v in constraints.items():
#     v[1].sort()

print(constraints)    # each value represents the --> 
                   # (agent with conflict, vertex number)

{0: [(4, 10), (7, 4), (3, 4), (3, 3), (3, 2)], 1: [(5, 7), (7, 0), (5, 6), (2, 8), (3, 1), (5, 5), (2, 7), (5, 4), (2, 6), (4, 3), (5, 3), (6, 4)], 2: [(4, 7), (4, 6), (6, 1), (7, 7), (4, 5), (6, 2), (4, 4), (5, 2), (6, 3), (1, 8), (4, 3), (5, 3), (6, 4), (1, 7), (5, 4), (1, 6), (3, 1), (5, 5), (3, 0)], 3: [(2, 9), (1, 6), (2, 8), (5, 5), (0, 6), (0, 5), (0, 4), (7, 2)], 4: [(6, 7), (6, 6), (6, 5), (1, 8), (2, 6), (5, 3), (6, 4), (2, 5), (5, 2), (6, 3), (2, 4), (6, 2), (2, 3), (6, 1), (7, 7), (2, 2), (0, 2)], 5: [(2, 5), (4, 4), (6, 3), (1, 8), (2, 6), (4, 3), (6, 4), (1, 7), (2, 7), (1, 6), (2, 8), (3, 1), (1, 5), (1, 4), (7, 0)], 6: [(7, 6), (2, 3), (4, 6), (7, 7), (2, 4), (4, 5), (2, 5), (4, 4), (5, 2), (1, 8), (2, 6), (4, 3), (5, 3), (4, 2), (4, 1), (4, 0)], 7: [(1, 4), (5, 7), (3, 5), (0, 3), (6, 0), (2, 3), (4, 6), (6, 1)]}


## Leave this cell for now

In [13]:
# Extracting first constraint for king_i 
# i = 0
# constraints[i][0] 

# Create a list of kings
all_kings = [None] * total_no_kings
all_kings_obs = {}

for k in range(total_no_kings):
    all_kings_obs[f'king_{k}_obstacles'] = obstacles


# Appending the first constraint for king_i's obstacle list
for k in range(total_no_kings):
    conflict_w_king = constraints[k][0][0]
    conflict_at_ver = constraints[k][0][1]
    conflict_vertex = init_paths[conflict_w_king][conflict_at_ver]
    all_kings_obs[f'king_{k}_obstacles'].append(conflict_vertex)
    # print(all_kings_obs[f'king_{k}_obstacles'][-1])     

In [30]:
## Check for the nodes conflicting at the saame vertex for first time-step

king_no = 0
time_step = 0

conflicting_node = init_paths[constraints[king_no][time_step][0]][constraints[king_no][time_step][1]]
# print(conflicting_node)

all_conflict_nodes = {}

for king_no in constraints:
    for const in constraints[king_no]:
        # print(king_no, const)
        conflicting_node = init_paths[const[0]][const[1]]
        print(conflicting_node)

(0, 5)
(1, 5)
(2, 5)
(3, 5)
(3, 4)
(4, 6)
(4, 6)
(4, 5)
(4, 4)
(4, 4)
(4, 4)
(4, 3)
(4, 3)
(4, 2)
(4, 2)
(4, 2)
(4, 2)
(0, 2)
(1, 2)
(1, 2)
(1, 2)
(2, 2)
(2, 2)
(3, 2)
(3, 2)
(3, 2)
(4, 2)
(4, 2)
(4, 2)
(4, 2)
(4, 3)
(4, 3)
(4, 4)
(4, 4)
(4, 4)
(5, 4)
(5, 4)
(4, 4)
(4, 4)
(4, 4)
(3, 4)
(3, 5)
(2, 5)
(2, 6)
(7, 2)
(6, 2)
(5, 2)
(4, 2)
(4, 2)
(4, 2)
(4, 2)
(3, 2)
(3, 2)
(3, 2)
(2, 2)
(2, 2)
(1, 2)
(1, 2)
(1, 2)
(0, 2)
(0, 5)
(3, 2)
(3, 2)
(3, 2)
(4, 2)
(4, 2)
(4, 2)
(4, 2)
(4, 3)
(4, 3)
(4, 4)
(4, 4)
(4, 4)
(4, 5)
(4, 6)
(4, 6)
(1, 3)
(1, 2)
(1, 2)
(1, 2)
(2, 2)
(2, 2)
(3, 2)
(3, 2)
(3, 2)
(4, 2)
(4, 2)
(4, 2)
(4, 2)
(5, 2)
(6, 2)
(7, 2)
(4, 6)
(4, 6)
(2, 6)
(1, 5)
(1, 3)
(1, 2)
(1, 2)
(1, 2)
